In [1]:
# 한글 사용 plt 설치 > https://github.com/ychoi-kr/koreanize-matplotlib 참조
!pip install koreanize-matplotlib

# facebook prophet 모델 설치 > https://github.com/facebook/prophet
!python -m pip install prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 43.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from prophet import Prophet # facebook prophet 모델
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib # 한글 사용을 위한 import
from sklearn.impute import KNNImputer # 결측치 
import os

import warnings
warnings.filterwarnings(action='ignore') 

In [21]:
train_path = '/content/drive/MyDrive/공모전/dataset/TRAIN/'
forders= os.listdir(train_path)

# 신흥동
data = pd.read_csv(train_path + forders[0])
data.head(2)

,연도,일시,측정소,PM2.5
0,0,01-01 00:00,신흥동,0.044
1,0,01-01 01:00,신흥동,0.064


In [22]:
test_path = '/content/drive/MyDrive/공모전/dataset/TEST_INPUT/'
forders= os.listdir(test_path)

# 신흥동
test = pd.read_csv(test_path + forders[4])
test.head(2)

,연도,일시,측정소,PM2.5
0,4,01-01 00:00,신흥동,0.056
1,4,01-01 01:00,신흥동,0.068


In [23]:
# 세종연서
aws_path = '/content/drive/MyDrive/공모전/dataset/TRAIN_AWS/'
forders= os.listdir(aws_path)

weather = pd.read_csv(aws_path + forders[23])
weather.head(2)

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,세종연서,0.173776,0.011389,0.025575,0.0,0.741
1,0,01-01 01:00,세종연서,0.173776,0.943889,0.023018,0.0,0.737


In [24]:
# 세종연서
aws_path = '/content/drive/MyDrive/공모전/dataset/TEST_AWS/'
forders= os.listdir(aws_path)

# cnt = 0
# for forder in forders :
#     print(f'{cnt} {forder}')
#     cnt += 1

test_weather = pd.read_csv(aws_path + forders[28])
test_weather.head(2)

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,4,01-01 00:00,세종연서,0.214850,0.104167,0.025575,0.0,0.564
1,4,01-01 01:00,세종연서,0.197472,0.937222,0.012788,0.0,0.559


In [25]:
# 결측치 채우기 - PM2.5
# KNNImputer 객체 생성
col = ['PM2.5']
imputer = KNNImputer()

# PM2.5열 선택
df = data[col]

# KNNImputer 객체 생성
imputer = KNNImputer()

# 결측치 채우기
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=col)

# 결과 출력
data[col] = df_imputed
print(data.isna().sum())

연도       0
일시       0
측정소      0
PM2.5    0
dtype: int64


In [26]:
# 결측치 채우기 - 기온(°C), 풍향(deg), 풍속(m/s), 강수량(mm), 습도(%)
# KNNImputer 객체 생성
cols = ['기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)']
imputer = KNNImputer()

# 기온(°C), 풍향(deg), 풍속(m/s), 강수량(mm), 습도(%) 열 선택
df = weather[cols]

# KNNImputer 객체 생성
imputer = KNNImputer()

# 결측치 채우기
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=cols)

# 결과 출력
weather[cols] = df_imputed
print(weather.isna().sum())

연도         0
일시         0
지점         0
기온(°C)     0
풍향(deg)    0
풍속(m/s)    0
강수량(mm)    0
습도(%)      0
dtype: int64


In [27]:
# prophet 모델 API
# https://facebook.github.io/prophet/docs/quick_start.html

# Prophet 모델 객체 생성
model = Prophet()

# 모델 학습
model.fit(data[['일시', 'PM2.5']].rename(columns={'일시': 'ds', 'PM2.5': 'y'}))

# test 데이터 읽어오기
test_df = test.copy()

# 예측 기간 설정
future = model.make_future_dataframe(periods=72, freq='H', include_history=False)

# 예측 수행
forecast = model.predict(future)

# 결과 출력
print(forecast[['ds', 'yhat']].tail(72))


OutOfBoundsDatetime: ignored

###facebook prophat 
 - https://facebook.github.io/prophet/docs/quick_start.html

###prophet 최신버전 사용이유
https://jayheya.tistory.com/85